In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import dotenv
import re
import pandas as pd
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, root_mean_squared_error, mean_absolute_error

## Initial Analysis
KielRegion has provided us with sample data ebfore we get access to the api. This notebook incudes the first analysis of this data.

In [5]:
df_hubs = pd.read_csv('Hub List all time + pick up and drop offs.csv')
df_pickup = pd.read_csv('2024 - Hub ID & Drop Off - Pick Up numbers.csv',sep=";")
df_loc = pd.read_csv('response_IDs_KielRegion.csv',sep=",")

In [6]:
# ids provided to us by Addix, since there is no way to request the station
# some IDs are dead
ids = [
    30467, 30466, 30464, 30434, 30420, 30390, 30282, 30207, 30185,
    30156, 30155, 30151, 30147, 30146, 30145, 30126, 30121, 30120,
    30095, 30084, 30027, 30026, 30025, 30024, 30023, 30022, 30011,
    29982, 29974, 29965, 29964, 29963, 29946, 29942, 29941, 29940,
    29939, 29938, 29937, 29936, 29935, 29934, 29933, 29745, 29663,
    29347, 29346, 29345, 29253, 29224, 29200, 29198, 29147, 29112,
    29109, 29083, 29049, 28932, 28814, 28813, 28812, 28811, 28804,
    28803, 28802, 28801, 28800, 28799, 28798, 28797, 28796, 28795,
    28794, 28793, 28792, 28791, 28790, 28789, 28749, 28738, 28264,
    27717, 27567, 27452, 27451, 27450, 27449, 27448, 27447, 27446,
    27445, 27444, 27443, 27442, 27441, 27440, 27382, 27381, 27380,
    27379, 27378, 27377, 27376, 27375, 27374, 27373, 27372, 27209,
    27183, 26889, 26805, 26730, 26725, 26724, 26708, 26653, 26224,
    26209, 24602, 24482, 24481, 24479, 24478, 24477, 24476, 24475,
    24474, 24473, 24472, 24471, 24470, 24469, 24468, 24467, 24466,
    24465, 24464, 24463, 24462, 24461, 24460, 24459, 24458, 24457,
    24456, 24455, 24454, 24453, 24452, 24451, 24450, 24449, 24448,
    24447, 24446, 24445, 24444, 24443, 24442, 24441, 24440, 24439,
    24438, 24437, 24436, 24435, 24434, 24433, 24432, 24431, 24430,
    24429, 24428, 24427, 24426, 24425, 24424, 24423, 24422, 24421,
    24420, 24419, 24418, 24417, 24416, 24415, 24414, 24413, 24412,
    24411, 24410, 24409, 24408, 24407, 24406, 24405, 24404, 24403,
    24402, 24401, 24400, 24399, 24398, 24397, 24396, 24395, 24394,
    24393, 24392, 24391, 24390, 24389, 24388, 24387, 24386, 24385,
    24384, 24383, 24381, 24380, 24379, 24378, 24377, 24376, 24375,
    24374, 24373, 24372, 24371, 24370, 24369, 24368, 24367
]

In [7]:
# simple adjustments to dataframe and its values
df_pickup.rename(columns = {'hub_id':'id'}, inplace=True)
df_combined = pd.merge(df_pickup, df_hubs, on='id')
df_combined["pick_drop_relation"] = round(df_combined["dropoffs"] /df_combined["pickups"],4)
df_combined["pick_drop_deficit"] = df_combined["# of unique dropoff_hub_id"] - df_combined["# of unique pickup_hub_id"]
df_combined['Days in full_date'] = pd.to_datetime(df_combined['Days in full_date'],format='%d/%m/%Y')

In [8]:
print(df_hubs.shape)
print(df_pickup.shape)
print(df_combined.shape)

(233, 7)
(6930, 5)
(6707, 13)


In [9]:
#find discrepency where bikes dont regulate themselves
df_combined.sort_values("pick_drop_relation",ascending=True)
df_combined.groupby(by="name")[['pickups', "dropoffs", "pick_drop_relation", "pick_drop_deficit"]].mean().sort_values("pick_drop_relation", ascending=False)

,pickups,dropoffs,pick_drop_relation,pick_drop_deficit
name,,,,
"Wik, Kanalfähre - Busstation",2929.0,3033.0,1.0355,-0.250000
MEKUN Olympiahochhaus,15389.0,15721.0,1.0216,-0.221374
Wellingdorf Stadtteilzentrum,6035.0,6136.0,1.0167,-0.166667
Alter Markt,15439.0,15676.0,1.0154,-0.307692
Anleger Dietrichsdorf,4627.0,4683.0,1.0121,-0.155378
thyssenkrupp Marine Systems,11136.0,11255.0,1.0107,-0.129412
Hörnbad,9990.0,10091.0,1.0101,-0.231076
Tilsiter Platz,5658.0,5697.0,1.0069,-0.205426
Vinetaplatz,8225.0,8281.0,1.0068,-0.131687


## Dataframe IDA

In [10]:
print(f'The data lists {len(df_combined.id.unique())} unique ids and {len(df_combined.name.unique())} unique names.')
#time series of specific place
sandkrug_df = df_combined[df_combined['name'] == 'Sandkrug'].sort_values("Days in full_date", ascending=False)

The data lists 36 unique ids and 36 unique names.


In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sandkrug_df['Days in full_date'], y=sandkrug_df['pick_drop_deficit'], mode='lines', name='Value'))
fig.update_layout(title='Time Series Line Plot', xaxis_title='Date', yaxis_title='Value')
fig.show()

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sandkrug_df['Days in full_date'], y=sandkrug_df['# of unique dropoff_hub_id'], mode='lines', name='Dropoff'))
fig.add_trace(go.Scatter(x=sandkrug_df['Days in full_date'], y=sandkrug_df['# of unique pickup_hub_id'], mode='lines', name='Pickup'))
fig.update_layout(title='Time Series Line Plot', xaxis_title='Date', yaxis_title='Value')
fig.show()

In [13]:
Wik_bus_df = df_combined[df_combined['name'] == ' Wik, Kanalfähre - Busstation'].sort_values("Days in full_date", ascending=False)
fig = go.Figure()
fig.add_trace(go.Scatter(x=Wik_bus_df['Days in full_date'], y=Wik_bus_df['# of unique dropoff_hub_id'], mode='lines', name='Dropoff'))
fig.add_trace(go.Scatter(x=Wik_bus_df['Days in full_date'], y=Wik_bus_df['# of unique pickup_hub_id'], mode='lines', name='Pickup'))
fig.update_layout(title='Time Series Line Plot', xaxis_title='Date', yaxis_title='Value')
fig.show()

# API
API ACCESS, that didn't work for the longest time because I was not given access till months later. First we request a bearer token, then use the token to request data.

In [14]:
# .env
config = dotenv.dotenv_values("paul_sprotte.env")

PASSWORD = config["PASSWORD"]

CLIENT_SECRET = config["CLIENT_SECRET"]

In [15]:
token_url = 'https://accounts.kielregion.addix.io/realms/infoportal/protocol/openid-connect/token'
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

data = {
    'grant_type': 'password',
    'username': 'business.brodmapa@outlook.com', 
    'password': PASSWORD,
    'client_id': 'quantumleap',
    'client_secret': 'fP81XZ5OTt5iRJ7qhyyTCv4eQtpGqc5i'
}

response = requests.post(token_url, headers=headers, data=data)
if response.status_code == 200:
    token_data = response.json()
    access_token = token_data['access_token']
    ACCESS_TOKEN = token_data['access_token']
    print(f"Bearer Token successful requested")
    if access_token:
        dotenv.set_key('paul_sprotte.env', 'ACCESS_TOKEN', access_token)
        print(f"Access Token erfolgreich in die .env-Datei geschrieben.")
        print(access_token)
else:
    print(f"Error: {response.status_code}, {response.text}")

Bearer Token successful requested
Access Token erfolgreich in die .env-Datei geschrieben.
eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJPbWpOY21GRmgwTjV6Wlg1eHg5Zk9mbE0xNFljTmY3WFlieG02OTJtczhBIn0.eyJleHAiOjE3MzI3NTk3NjQsImlhdCI6MTczMjYxNTc2NCwianRpIjoiY2ZjYzhlNGItYTdkZC00ZjI2LTg4ZDMtYzdlMzRiNDVhMjVmIiwiaXNzIjoiaHR0cHM6Ly9hY2NvdW50cy5raWVscmVnaW9uLmFkZGl4LmlvL3JlYWxtcy9pbmZvcG9ydGFsIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6IjI0NzQ4NTgxLTU4MGEtNDljZS1iZmY2LTY0YmMwYmExYjY1ZiIsInR5cCI6IkJlYXJlciIsImF6cCI6InF1YW50dW1sZWFwIiwic2Vzc2lvbl9zdGF0ZSI6IjZhMzIyNjA3LTJmMTEtNDUyYy1iMDg1LTY2MThiMzA1ZThlMiIsImFjciI6IjEiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy1pbmZvcG9ydGFsIiwib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7InF1YW50dW1sZWFwIjp7InJvbGVzIjpbImRhdGEtY29uc3VtZXItcWwiXX0sImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoicm9sZXMgcHJvZmlsZSBlbWFpbCIsInNpZCI6IjZhMzIyNjA3LTJmMTEtNDUyYy1iMDg1LT

In [16]:
params = {
    'type': 'BikeHireDockingStation',
    'fromDate': '2024-11-13T00:00:00',
    'toDate': '2024-11-13T23:59:59',
    'attrs': 'name,totalSlotNumber,freeSlotNumber,Location'
}

header = {
    'NGSILD-Tenant': 'infoportal',
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}


collection_responses = []
station_ids = []
for curr_id in ids:
    url = f"https://apis.kielregion.addix.io/ql/v2/entities/urn:ngsi-ld:BikeHireDockingStation:KielRegion:{curr_id}"
    response = requests.get(url, headers=header, params=params)

    if response.status_code == 200:
        response_data = response.json()
        print(f'got a response - {curr_id}')
        collection_responses.append(response_data)
        station_ids.append(curr_id)
    else:
        print(f"Error: {response.status_code}, for {curr_id}, {response.text}", end='\r')

Error: 404, for 30467, {
  "description": "No records were found for such query.",
  "error": "Not Found"
}
Error: 404, for 30466, {
  "description": "No records were found for such query.",
  "error": "Not Found"
}
Error: 404, for 30464, {
  "description": "No records were found for such query.",
  "error": "Not Found"
}
got a response - 30434
Error: 404, for 30420, {
  "description": "No records were found for such query.",
  "error": "Not Found"
}
Error: 404, for 30390, {
  "description": "No records were found for such query.",
  "error": "Not Found"
}
Error: 404, for 30282, {
  "description": "No records were found for such query.",
  "error": "Not Found"
}
got a response - 30207
Error: 404, for 30185, {
  "description": "No records were found for such query.",
  "error": "Not Found"
}
Error: 404, for 30156, {
  "description": "No records were found for such query.",
  "error": "Not Found"
}
Error: 404, for 30155, {
  "description": "No records were found for such query.",
  "erro

In [17]:
print(len(ids))
print(len(collection_responses))

233
196


In [18]:
collection_responses[0]['attributes'][2]['values'][0]

'Felde Gemeindezentrum'

In [22]:
max_capacity = []
station_name = []
lat = []
long = []
for curr_result in collection_responses:
    lat.append(curr_result['attributes'][1]['values'][0]['coordinates'][0])
    long.append(curr_result['attributes'][1]['values'][0]['coordinates'][1])
    max_capacity.append(curr_result['attributes'][3]['values'][0])
    station_name.append(curr_result['attributes'][2]['values'][0])


In [24]:
df = pd.DataFrame(columns=['entityId','station_name', 'maximum_capacity', 'longitude', 'latitude', 'subarea'])
df['entityId'] = station_ids
df['maximum_capacity'] = max_capacity
df['station_name'] = station_name
df['longitude'] = long
df['latitude'] = lat

In [26]:
df.drop_duplicates(inplace=True)
df = df[df['entityId'] != 24470]
print(df.shape)
df.to_csv("station_data.csv", index=False)

(195, 6)


In [ ]:
df = df[df['entity_id'] != 30011]

,entity_id,station_name,max_capacity,longitude,latitude,sub_area
91,24472,Ascheberger Parkplatz,20,54.152798,10.404486,NaN


# Analysis

In [ ]:
df_flx = pd.read_csv('FelixData_final_main_kiel.csv')

In [ ]:
print(f'''The dataframe has a {df_flx.shape[0]} rows on {df_flx.shape[1]} columns. The colums
have the following names: \n{'\n'.join(df_flx.columns.tolist())}''')

The dataframe has a 86501 rows on 9 columns. The colums
have the following names: 
index
entityId
entityType
name
availableBikeNumber
freeSlotNumber
totalSlotNumber
pickups
dropoffs


## Visualisation

In [ ]:
# further visualisation could be about
# analyse these in terms of: 
# - how long does a station stay empty/overcrowded
# - how intense is the overcpopulation
# - which stations are regular for overpopulation
# - relation between weather and bike requests
df_flx[df_flx['availableBikeNumber'] == 0]
df_flx[df_flx['availableBikeNumber'] > df_flx['totalSlotNumber']]

,index,entityId,entityType,name,availableBikeNumber,freeSlotNumber,totalSlotNumber,pickups,dropoffs
4570,2024-09-20T11:00:00.000+00:00,24368,BikeHireDockingStation,Umsteiger,20.083333,0.283333,20.0,7633,7629
4571,2024-09-20T12:00:00.000+00:00,24368,BikeHireDockingStation,Umsteiger,20.116667,0.583333,20.0,7633,7629
4572,2024-09-20T13:00:00.000+00:00,24368,BikeHireDockingStation,Umsteiger,21.050000,0.266667,20.0,7633,7629
4574,2024-09-20T15:00:00.000+00:00,24368,BikeHireDockingStation,Umsteiger,21.650000,0.650000,20.0,7633,7629
4575,2024-09-20T16:00:00.000+00:00,24368,BikeHireDockingStation,Umsteiger,21.203390,1.169492,20.0,7633,7629
...,...,...,...,...,...,...,...,...,...
83936,2024-09-24T16:00:00.000+00:00,26224,BikeHireDockingStation,"Wik, Kanalfähre - Busstation",20.700000,0.000000,20.0,2929,3033
83937,2024-09-24T17:00:00.000+00:00,26224,BikeHireDockingStation,"Wik, Kanalfähre - Busstation",21.000000,0.000000,20.0,2929,3033
83938,2024-09-24T18:00:00.000+00:00,26224,BikeHireDockingStation,"Wik, Kanalfähre - Busstation",21.000000,0.000000,20.0,2929,3033
84082,2024-09-30T18:00:00.000+00:00,26224,BikeHireDockingStation,"Wik, Kanalfähre - Busstation",20.500000,0.300000,20.0,2929,3033


In [ ]:
df_flx.name.unique()

array(['Anleger Dietrichsdorf', 'Umsteiger', 'Alte Mu/Brunswiker Str.',
       'Alter Markt', 'Anleger Reventlou', 'Christian-Albrechts-Platz',
       'Hörnbad', 'Kirchhofallee', 'RBZ Wirtschaft', 'Vinetaplatz',
       'Westring/Eckernförder Str.', 'CAU Sportstätten', 'Dreiecksplatz',
       'Hansastr./Gutenbergstr.', 'Blücherplatz', 'Andreas-Gayk-Straße',
       'Tilsiter Platz', 'Wellingdorf Stadtteilzentrum', 'Hauptbahnhof',
       'Exerzierplatz', 'thyssenkrupp Marine Systems',
       'MEKUN Olympiahochhaus', 'Bernhard-Minetti-Platz',
       'KVG Betriebshof Diedrichstraße', 'Sophienhof', 'ADAC-Station',
       'Förde Sparkasse Lorentzendamm', 'Studiale', 'UKSH', 'Sandkrug',
       'Zur Fähre', 'Seefischmarkt', 'Krausstraße',
       'Kieler Innovations- und Technologiezentrum',
       ' Wik, Kanalfähre - Busstation', 'Wilhelmplatz'], dtype=object)

## ML Part
We will look at the Holt Winters Model, that focuses on reoccuring patterns in the data:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error


In [ ]:
def reform_dataframe(df_, location_): 
    df_ags = df_[df_['name'] == location_]
    df_tmp = df_ags.copy()

    df_tmp['index'] = pd.to_datetime(df_tmp['index']) #more features

    # Features erzeugen
    df_tmp['Year'] = df_tmp['index'].dt.year
    df_tmp['Month'] = np.sin(df_tmp['index'].dt.month)
    df_tmp['Day'] = df_tmp['index'].dt.day
    df_tmp['Hour'] = df_tmp['index'].dt.hour  # Stunde des Tages im 24-Stunden-Format
    df_tmp['Season'] = df_tmp['index'].dt.month % 12 // 3 + 1  # Jahreszeit (1=Winter, 2=Frühling, 3=Sommer, 4=Herbst)
    df_tmp['Weekend'] = df_tmp['index'].dt.weekday >= 5  # Boolescher Wert für Wochenende (Samstag=5, Sonntag=6)
    df_tmp['DayOfWeek'] = np.cos(df_tmp['index'].dt.dayofweek)
    df_tmp['DayOfYear'] = df_tmp['index'].dt.dayofyear

    df_tmp.dropna(inplace=True)
    df_tmp.set_index('index', inplace=True)
    df_tmp.dropna(inplace=True)
    df_tmp.index.tz_localize(None)
    df_tmp = df_tmp.asfreq('h')

    return df_tmp

df_ags_mf = reform_dataframe(df_flx, 'Alter Markt')

In [ ]:
def plot_that(data1_, forcast_):
    from plotly.subplots import make_subplots
    fig = make_subplots()
    orng = '#DF7D39'
    lbl = '#66BECF'
    color = '#092947'

    fig.add_trace(go.Scatter(x=data1_.index, y=data1_ ,name='totalSlotNumber',
        marker=dict(color=lbl),),
                row=1, col=1)
    fig.add_trace(go.Scatter(x=forcast_.index, y=forcast_ ,name='Forecast',
        marker=dict(color=orng),),
                row=1, col=1)
    fig.update_traces(textposition="bottom right")
    fig.update_layout(
        width = 1000,
        height = 500,
        title="Besetzungsstsatus der Radstation",
        yaxis_title="Verfügbare Räder",
        xaxis_title="Datum",
        paper_bgcolor='white',  # Background of the whole plot
        plot_bgcolor='white',
        xaxis=dict(
            gridwidth=1,                # Width of vertical grid lines
            showline=True,
            linewidth=2, linecolor=color
        ),
        yaxis=dict(
            #range=[0, 300],  # Set the y-axis limits from 0 to 10
            showgrid=True,             # Show horizontal grid lines
            gridcolor='lightgrey',         # Color of horizontal grid lines
            gridwidth=1,                # Width of horizontal grid lines
            showline=True,
            linewidth=2, linecolor=color
        )
    )
    fig.show()

In [ ]:
train = df_ags_mf[(df_ags_mf.index <= '2024-08-25 00:00:00+00:00') & (df_ags_mf.index > '2024-08-08 00:00:00+00:00')]['availableBikeNumber']
test = df_ags_mf[(df_ags_mf.index >= '2024-08-25 00:00:00+00:00') & (df_ags_mf.index < '2024-08-26 00:00:00+00:00')]['availableBikeNumber']
try:
    model = ExponentialSmoothing(train, trend='add', seasonal='add', initialization_method = 'estimated',
                                 seasonal_periods=24)
    fit = model.fit(smoothing_seasonal=0.2,)
    forecast = fit.forecast(
        steps=24)
except:
    print(fit.mle_retvals)
print(f'Mean Squared Error: {mean_squared_error(test, forecast)}')
print(f'Mean Absolute percentage Error: {mean_absolute_percentage_error(test, forecast)}')
print(f'Root Mean Squared Error: {root_mean_squared_error(test, forecast)}')
print(f'Root Absolute Error: {mean_absolute_error(test, forecast)}')
plot_that(test,forecast)

Mean Squared Error: 33.73052639110497
Mean Absolute percentage Error: 0.26649256407557437
Root Mean Squared Error: 5.807798756078328
Root Absolute Error: 4.4897997028324195


Initially we check if we could simply predict the avaiable bikes over the course of a day to see how well the prediction would go well. After around 5h the data is not realistic anymore.

---

In [ ]:
train = df_ags_mf[(df_ags_mf.index <= '2024-08-21 00:00:00+00:00') & (df_ags_mf.index > '2024-08-9 00:00:00+00:00')]['availableBikeNumber']
test = df_ags_mf[(df_ags_mf.index >= '2024-08-21 00:00:00+00:00') & (df_ags_mf.index < '2024-08-23 00:00:00+00:00')]['availableBikeNumber']
try:
    model = ExponentialSmoothing(train, trend='add', seasonal='add', initialization_method = 'estimated',
                                 seasonal_periods=24)
    fit = model.fit(smoothing_seasonal=0.2,)
    forecast = fit.forecast(
        steps=48)
except:
    print(fit.mle_retvals)
print(f'Mean Squared Error: {mean_squared_error(test, forecast)}')
print(f'Mean Absolute percentage Error: {mean_absolute_percentage_error(test, forecast)}')
print(f'Root Mean Squared Error: {root_mean_squared_error(test, forecast)}')
plot_that(test,forecast)

Mean Squared Error: 1.1283807550727512
Mean Absolute percentage Error: 1352852910475095.5
Root Mean Squared Error: 1.0622526794848537


---

Now we will attempt to use as much data before as we can to predict the following 5 hours.

In [ ]:
try:
    model = ExponentialSmoothing(df_ags_mf[df_ags_mf.index >= '2024-09-25 00:00:00+00:00']['availableBikeNumber'], 
                                trend='add', 
                                seasonal='add', 
                                seasonal_periods=48)
    fit = model.fit(smoothing_seasonal=0.2,)#smoothing_level=1, smoothing_slope=1, smoothing_seasonal=1)
    forecast = fit.forecast(steps=12)
except:
    print(fit.mle_retvals)
plot_that(df_ags_mf[df_ags_mf.index >= '2024-09-25 00:00:00+00:00']['availableBikeNumber'],forecast)

## Secondary testing
instead of a single step the following section will iterate either 1h at a time and predict it, or do the same with 5 hours.

In [ ]:
def test_iterate(df_, start_date_, train_start_date, end_date_, end_test_, steps_ = '1h'): #1h or 5h
    tme_steps_ = pd.date_range(start=start_date_, end=end_date_, freq=steps_)
    aggregated_forecasts = pd.Series()
    for curr_stp in tme_steps_:
        df_.dropna(inplace=True)
        df_cutout_ = df_.loc[train_start_date:curr_stp]
        try:
            model = ExponentialSmoothing(df_cutout_['availableBikeNumber'], 
                                        trend='add', 
                                        seasonal='add', 
                                        freq='h',
                                        seasonal_periods=24)
            fit = model.fit(smoothing_seasonal=0.2)
            forecast_ = fit.forecast(steps=5)
            aggregated_forecasts = aggregated_forecasts.add(forecast_, fill_value=0)
        except:
            print(fit.mle_retvals)
    plot_that(df_.loc[start_date_:end_test_]['availableBikeNumber'], aggregated_forecasts)
    return aggregated_forecasts, df_.loc[start_date_:end_test_]

In [ ]:
train_start_date = pd.Timestamp('2024-06-17 11:00:00+0000', tz='UTC')
start_date = pd.Timestamp('2024-06-20 23:00:00+0000', tz='UTC')
end_date = pd.Timestamp('2024-07-29 18:00:00+00:00', tz='UTC')
end_test = pd.Timestamp('2024-07-29 23:00:00+00:00', tz='UTC')

tsts_ls = []
forcasts = []
loc_lis = ['Alter Markt','Anleger Dietrichsdorf', 'Hauptbahnhof']
for location in loc_lis:
    curr_df = reform_dataframe(df_flx, location)
    frcst_, tsts = test_iterate(curr_df, start_date,train_start_date, end_date,end_test, '5h')
    forcasts.append(frcst_)
    tsts_ls.append(tsts)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 223.8904704832064
       x: [ 1.000e+00  3.009e-09 ...  9.407e-02  2.089e-01]
     nit: 16
     jac: [-2.938e-01  2.294e+02 ...  1.665e-03  8.926e-04]
    nfev: 484
    njev: 16
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 223.8904704832064
       x: [ 1.000e+00  3.009e-09 ...  9.407e-02  2.089e-01]
     nit: 16
     jac: [-2.938e-01  2.294e+02 ...  1.665e-03  8.926e-04]
    nfev: 484
    njev: 16
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 223.8904704832064
       x: [ 1.000e+00  3.009e-09 ...  9.407e-02  2.089e-01]
     nit: 16
     jac: [-2.938e-01  2.294e+02 ...  1.665e-03  8.926e-04]
    nfev: 484
    njev: 16
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 223.8904704832064
       x: [ 1.000e+00  3.009e-09 ...  9.407e-02  2.089e-01]
     nit: 16
     jac: [-2.938e-01  2.294e+02 ...  1

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 2031.4404899612596
       x: [ 1.000e+00  2.205e-07 ... -1.613e+00 -1.503e+00]
     nit: 24
     jac: [-6.252e+02  2.025e+03 ...  1.421e-02  8.774e-03]
    nfev: 602
    njev: 20
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 2031.4404899612596
       x: [ 1.000e+00  2.205e-07 ... -1.613e+00 -1.503e+00]
     nit: 24
     jac: [-6.252e+02  2.025e+03 ...  1.421e-02  8.774e-03]
    nfev: 602
    njev: 20
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 2031.4404899612596
       x: [ 1.000e+00  2.205e-07 ... -1.613e+00 -1.503e+00]
     nit: 24
     jac: [-6.252e+02  2.025e+03 ...  1.421e-02  8.774e-03]
    nfev: 602
    njev: 20
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 2031.4404899612596
       x: [ 1.000e+00  2.205e-07 ... -1.613e+00 -1.503e+00]
     nit: 24
     jac: [-6.252e+02  2.025e+03 ..

For some reason some of the dataframes only display a small time range. This isn't intended. What is noticable is a) over such a long time frame the prediction looks promising when using a 24h period, which with the holt Winters model of course focuses mostly on a pattern that repeats itself. This is where a singificant problem with this model lay. The prediction pattern repeated itself even if the actual data did not. This, combined with seasonal changes and the lack of other data to input lead to predictions we deemed: too simplistic.

In [ ]:
curr_df = reform_dataframe(df_flx, 'Hauptbahnhof')

In [ ]:
train_start_date = pd.Timestamp('2024-09-01 00:00:00+00:00', tz='UTC')
start_date = pd.Timestamp('2024-09-10 00:00:00+00:00', tz='UTC')
end_date = pd.Timestamp('2024-09-30 16:00:00+0000', tz='UTC')
end_test = pd.Timestamp('2024-09-30 19:00:00+0000', tz='UTC')

tsts_ls = []
forcasts = []
loc_lis = ['Alter Markt','Anleger Dietrichsdorf', 'Hauptbahnhof']
for location in loc_lis:
    curr_df = reform_dataframe(df_flx, location)
    frcst_, tsts = test_iterate(curr_df, start_date, train_start_date, end_date,end_test, '5h')
    forcasts.append(frcst_)
    tsts_ls.append(tsts)

Similar to the promising results for the bike station, the main station is a very busy place, which helps the holt winters model. However similar to the previous station test, the 'small differences' worked out to over 10 bikes difference and would sometimes miss the times of alert for both over and under representation. Considering that testing has shown the severe issues the model faces when confronted with emptier stations, it did do more than it should and that was expected of it.

In [ ]:
tsts_ls[0]

,entityId,entityType,name,availableBikeNumber,freeSlotNumber,totalSlotNumber,pickups,dropoffs,Year,Month,Day,Hour,Season,Weekend,DayOfWeek,DayOfYear
index,,,,,,,,,,,,,,,,
2024-09-10 00:00:00+00:00,24370.0,BikeHireDockingStation,Alter Markt,1.000000,25.000000,25.0,15439.0,15676.0,2024.0,0.412118,10.0,0.0,4.0,False,0.540302,254.0
2024-09-10 01:00:00+00:00,24370.0,BikeHireDockingStation,Alter Markt,1.000000,25.000000,25.0,15439.0,15676.0,2024.0,0.412118,10.0,1.0,4.0,False,0.540302,254.0
2024-09-10 02:00:00+00:00,24370.0,BikeHireDockingStation,Alter Markt,0.133333,25.000000,25.0,15439.0,15676.0,2024.0,0.412118,10.0,2.0,4.0,False,0.540302,254.0
2024-09-10 03:00:00+00:00,24370.0,BikeHireDockingStation,Alter Markt,0.333333,25.000000,25.0,15439.0,15676.0,2024.0,0.412118,10.0,3.0,4.0,False,0.540302,254.0
2024-09-10 04:00:00+00:00,24370.0,BikeHireDockingStation,Alter Markt,0.633333,24.950000,25.0,15439.0,15676.0,2024.0,0.412118,10.0,4.0,4.0,False,0.540302,254.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-30 15:00:00+00:00,24370.0,BikeHireDockingStation,Alter Markt,20.650000,5.333333,25.0,15439.0,15676.0,2024.0,0.412118,30.0,15.0,4.0,False,1.000000,274.0
2024-09-30 16:00:00+00:00,24370.0,BikeHireDockingStation,Alter Markt,23.783333,1.800000,25.0,15439.0,15676.0,2024.0,0.412118,30.0,16.0,4.0,False,1.000000,274.0
2024-09-30 17:00:00+00:00,24370.0,BikeHireDockingStation,Alter Markt,24.333333,0.666667,25.0,15439.0,15676.0,2024.0,0.412118,30.0,17.0,4.0,False,1.000000,274.0


In [ ]:
results_lis = []
for result, test in zip (forcasts,tsts_ls):
    evals = {}
    print(len(result))
    print(f'Mean Squared Error: {mean_squared_error(test['availableBikeNumber'], result)}')
    evals['mse'] = mean_squared_error(test['availableBikeNumber'], result)
    print(f'Mean Absolute percentage Error: {mean_absolute_percentage_error(test['availableBikeNumber'], result)}')
    evals['MAPE'] = mean_absolute_percentage_error(test['availableBikeNumber'], result)
    print(f'Root Mean Squared Error: {root_mean_squared_error(test['availableBikeNumber'], result)}')
    evals['RMSE'] = root_mean_squared_error(test['availableBikeNumber'], result)
    results_lis.append(evals)

500
Mean Squared Error: 4.99429830169348
Mean Absolute percentage Error: 28592077256430.367
Root Mean Squared Error: 2.234792675326613
500
Mean Squared Error: 3.6900472234698283
Mean Absolute percentage Error: 95824235787000.05
Root Mean Squared Error: 1.9209495629687492
500
Mean Squared Error: 32.94755613062674
Mean Absolute percentage Error: 330404534358698.4
Root Mean Squared Error: 5.73999617862475


In [ ]:
df = pd.DataFrame(results_lis)

In [ ]:
df

,mse,MAPE,RMSE
0,4.994298,2.859208e+13,2.234793
1,3.690047,9.582424e+13,1.920950
2,32.947556,3.304045e+14,5.739996


These were the final measurements taken for the comparison of our different model trainings.

In [ ]:
df.MAPE.mean()

np.float64(151606949134042.9)